In [12]:
import numpy as np
import matplotlib.pyplot as plt

def generate_dummy_eeg_data(self, params, buffer_duration):
        # Extract parameters from JSON dictionary
        num_channels = params.get("eeg_channels", 8)
        samples_per_second = params.get("sampling_rate", 256)
        epoch_period = buffer_duration
        noise_level = params.get("noise", 1)
        artifact_prob = params.get("artifacts", 0.01)
        modulation_type = params.get("modulation_type", 'sinusoidal')
        preset = params.get("preset", None)
        sequence = params.get("sequence", None)
        correlation_strength = params.get("correlation_strength", 0.5)  # Strength of correlation between nearby channels
        power_law_slope = params.get("power_law_slope", 1.0)

        # Preset amplitude settings
        preset_settings = {
            #           del  the  alp  bet  gam
            'focus':   [0.1, 0.1, 0.5, 0.8, 0.4],
            'alert':   [0.1, 0.1, 0.4, 0.9, 0.3],
            'relaxed': [0.2, 0.2, 0.7, 0.3, 0.2],
            'drowsy':  [0.4, 0.6, 0.2, 0.2, 0.1],
        }

        if preset in preset_settings:
            delta_amp, theta_amp, alpha_amp, beta_amp, gamma_amp = preset_settings[preset]
        else:
            delta_amp = params.get("delta_amp", 0.1)
            theta_amp = params.get("theta_amp", 0.1)
            alpha_amp = params.get("alpha_amp", 0.1)
            beta_amp = params.get("beta_amp", 0.1)
            gamma_amp = params.get("gamma_amp", 0.1)
        
        total_samples = samples_per_second * epoch_period
        t = np.linspace(0, epoch_period, total_samples, endpoint=False)
        eeg_data = np.zeros((num_channels, total_samples))

        # Frequency bands
        bands = {
            'Delta': (0.5, 4),
            'Theta': (4, 8),
            'Alpha': (8, 13),
            'Beta': (13, 30),
            'Gamma': (30, 100)
        }

        amplitudes = {
            'Delta': delta_amp,
            'Theta': theta_amp,
            'Alpha': alpha_amp,
            'Beta': beta_amp,
            'Gamma': gamma_amp
        }

        # Managing the type of EEG modulation
        if modulation_type == 'sinusoidal':
            modulating_freq = 0.1  # frequency of the amplitude modulation
            delta_mod = (1 + np.sin(2 * np.pi * modulating_freq * t)) / 2  # between 0.5 and 1.5
            theta_mod = (1 + np.cos(2 * np.pi * modulating_freq * t)) / 2
            alpha_mod = (1 + np.sin(2 * np.pi * modulating_freq * t + np.pi / 4)) / 2
            beta_mod = (1 + np.cos(2 * np.pi * modulating_freq * t + np.pi / 4)) / 2
            gamma_mod = (1 + np.sin(2 * np.pi * modulating_freq * t + np.pi / 2)) / 2
        elif modulation_type == 'random':
            delta_mod = np.abs(np.random.randn(total_samples))
            theta_mod = np.abs(np.random.randn(total_samples))
            alpha_mod = np.abs(np.random.randn(total_samples))
            beta_mod = np.abs(np.random.randn(total_samples))
            gamma_mod = np.abs(np.random.randn(total_samples))
        
        for band, (low, high) in bands.items():
            amplitude = amplitudes[band]
            freqs = np.linspace(low, high, int(samples_per_second / 2))
            power_law = freqs ** -power_law_slope

            for i in range(num_channels):
                for f, p in zip(freqs, power_law):
                    phase = np.random.uniform(0, 2 * np.pi)
                    if band == 'Delta':
                        eeg_data[i] += amplitude * p * delta_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Theta':
                        eeg_data[i] += amplitude * p * theta_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Alpha':
                        eeg_data[i] += amplitude * p * alpha_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Beta':
                        eeg_data[i] += amplitude * p * beta_mod * np.sin(2 * np.pi * f * t + phase)
                    elif band == 'Gamma':
                        eeg_data[i] += amplitude * p * gamma_mod * np.sin(2 * np.pi * f * t + phase)

        # Adding random noise
        eeg_data += noise_level * np.random.randn(num_channels, total_samples)

        # Introducing correlation between nearby channels
        for channel in range(1, num_channels):
            eeg_data[channel] += correlation_strength * eeg_data[channel - 1]

        # Introducing artifacts as random peaks
        artifact_indices = np.random.choice(total_samples, int(artifact_prob * total_samples), replace=False)
        for channel in range(0, num_channels):
            eeg_data[channel, artifact_indices] -= np.random.uniform(10, 20, len(artifact_indices))

        # Handle sequence if provided
        if sequence:
            full_data = []
            for seq in sequence:
                preset, duration = seq
                temp_params = params.copy()
                temp_params['preset'] = preset
                temp_params['epoch_period'] = duration
                temp_params['sequence'] = None
                segment = generate_dummy_eeg_data(temp_params)
                full_data.append(segment)
            eeg_data = np.hstack(full_data)
            print(eeg_data)

        return eeg_data
                    
print(temp_params)
# Générer les ondes alpha sans modificateur noise
t, alpha1_no_noise = generate_alpha_wave(frequency, amplitude_alpha1, sampling_rate, duration)
_, alpha2_no_noise = generate_alpha_wave(frequency, amplitude_alpha2, sampling_rate, duration)

# Générer les ondes alpha avec modificateur noise
_, alpha1_with_noise = generate_alpha_wave(frequency, amplitude_alpha1, sampling_rate, duration, noise_level)
_, alpha2_with_noise = generate_alpha_wave(frequency, amplitude_alpha2, sampling_rate, duration, noise_level)

# Conditions
condition1 = amplitude_alpha1 == 0.1
condition2 = amplitude_alpha2 == 0.9

# Afficher les résultats
plt.figure(figsize=(12, 8))

# Alpha1 sans bruit
plt.subplot(2, 2, 1)
plt.plot(t, alpha1_no_noise)
plt.title(f'Alpha1 sans bruit (Amplitude = {amplitude_alpha1})')

# Alpha2 sans bruit
plt.subplot(2, 2, 2)
plt.plot(t, alpha2_no_noise)
plt.title(f'Alpha2 sans bruit (Amplitude = {amplitude_alpha2})')

# Alpha1 avec bruit
plt.subplot(2, 2, 3)
plt.plot(t, alpha1_with_noise)
plt.title(f'Alpha1 avec bruit (Amplitude = {amplitude_alpha1}, Bruit = {noise_level})')

# Alpha2 avec bruit
plt.subplot(2, 2, 4)
plt.plot(t, alpha2_with_noise)
plt.title(f'Alpha2 avec bruit (Amplitude = {amplitude_alpha2}, Bruit = {noise_level})')

plt.tight_layout()
plt.show()

# Comparaison des conditions
print(f"Condition 1 (Alpha1  0.1): {condition1}")
print(f"Condition 2 (Alpha2 > 0.9): {condition2}")

# Vérification des résultats avec et sans bruit
alpha1_meets_condition1 = np.max(np.abs(alpha1_no_noise)) > 0.1
alpha2_meets_condition2 = np.max(np.abs(alpha2_no_noise)) > 0.9

alpha1_with_noise_meets_condition1 = np.max(np.abs(alpha1_with_noise)) > 0.1
alpha2_with_noise_meets_condition2 = np.max(np.abs(alpha2_with_noise)) > 0.9

print("\nSans bruit:")
print(f"Alpha1 respecte la condition 1: {alpha1_meets_condition1}")
print(f"Alpha2 respecte la condition 2: {alpha2_meets_condition2}")

print("\nAvec bruit:")
print(f"Alpha1 respecte la condition 1: {alpha1_with_noise_meets_condition1}")
print(f"Alpha2 respecte la condition 2: {alpha2_with_noise_meets_condition2}")



NameError: name 'temp_params' is not defined

In [ ]:
from scipy import signal

# Define frequency bands
bands = {
    'Delta': (1, 4, 'yellow'),
    'Theta': (4, 8, 'green'),
    'Alpha': (8, 13, 'cyan'),
    'Beta': (13, 30, 'blue'),
    'Gamma': (30, 100, 'red')
}

fig, axes = plt.subplots(num_channels, 1, figsize=(15, 10), sharex=True)

if num_channels == 1:
    axes = [axes]

for i in range(num_channels):
    # Plotting the PSD
    freqs, psd = signal.welch(data[i], fs=samples_per_second)
    axes[i].semilogy(freqs, psd)
    axes[i].set_title(f'Channel {i+1} PSD')
    axes[i].set_ylabel('PSD (V^2/Hz)')
    axes[i].grid(True)

    # Highlighting frequency bands
    for band, (low, high,col) in bands.items():
        axes[i].axvspan(low, high, color=col, alpha=0.3)
        axes[i].text((low + high) / 2, max(psd), band, horizontalalignment='center', verticalalignment='top')

axes[-1].set_xlabel('Frequency (Hz)')
plt.tight_layout()
plt.show()